This notebook can be run on the Copernicus Dataspace Jupyterhub but running the following package installation cell first

**Note** You should select on of the kernels with GDAL installed, eg. "Geo science"

In [1]:
try:
    import eomaji
except ModuleNotFoundError:
    !pip install eomaji@git+https://github.com/DHI/EOMAJI-OpenEO-toolbox.git

# Meteorological & Biophysical Parameter Workflow

In [2]:
import openeo
from pathlib import Path
from eomaji.workflows.meteo_preprocessing import get_meteo_data
from eomaji.workflows.sentinel2_preprocessing import split_datasets_to_tiffs
from eomaji.workflows.prepare_data_cubes import prepare_data_cubes
from eomaji.utils.general_utils import read_area_date_info
from eomaji.utils.raster_utils import resample_to_s2

## 1. Set up the OpenEO connection

In [3]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

2025-06-19 14:42:39,750 [INFO] Loaded openEO client config from sources: []
2025-06-19 14:42:40,368 [INFO] Found OIDC providers: ['CDSE']
2025-06-19 14:42:40,372 [INFO] No OIDC provider given, but only one available: 'CDSE'. Using that one.
2025-06-19 14:42:40,765 [INFO] Using default client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e' from OIDC provider 'CDSE' info.
2025-06-19 14:42:40,773 [INFO] Found refresh token: trying refresh token based authentication.
2025-06-19 14:42:40,776 [INFO] Doing 'refresh_token' token request 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' with post data fields ['grant_type', 'client_id', 'refresh_token'] (client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e')
2025-06-19 14:42:41,035 [INFO] Obtained tokens: ['access_token', 'id_token', 'refresh_token']
2025-06-19 14:42:41,037 [INFO] Storing refresh token for issuer 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE' (client 'sh-b1c3a958-52d4-40fe-a333-153

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

## 2. Define AOI and date
Either read it from dumped information from the [prepare_data.ipynb](notebooks/prepare_data.ipynb) notebook, or define it yourself

In [4]:
date_dir = "./data"
date, bbox = read_area_date_info(
    dir=date_dir
)
# date = datetime.date(2023, 6, 25)
#bbox = [6.153142, 45.045924, 6.433234, 45.251259]

## 3. Download Sentinel 2 and Sentinel 3 data for AOI and date
**Note** If you this function checks if the data already exists first

In [ ]:
s2_path, s3_path, vza_path, worldcover_path, dem_s2_path, dem_s3_path, sentinel3_acq_time, = prepare_data_cubes(
    connection=connection,
    bbox=bbox,
    date=date,
    sentinel2_search_range = 3,
    out_dir = date_dir,
)

2025-06-19 14:42:41,220 [INFO] Cached Sentinel 2 data cube found. Skipping download.
/usr/local/lib/python3.10/dist-packages/openeo/rest/connection.py:1171: UserWarning: Property filtering with unsupported properties according to collection/STAC metadata: {'timeliness', 'orbitDirection'} (supported: dict_keys(['bands', 'eo:bands', 'raster:bands'])).
  return DataCube.load_collection(
2025-06-19 14:42:41,328 [INFO] Cached DEM data cube found. Skipping download.


2025-06-19 14:42:41,330 [INFO] Cached DEM cube found. Skipping download.
2025-06-19 14:42:41,398 [INFO] Cached Worldcover cube found. Skipping download.
2025-06-19 14:42:41,400 [INFO] Data cubes prepared and saved.


In [6]:
tif_path = split_datasets_to_tiffs(s2_path = s2_path, s3_path = s3_path, worldcover_path = worldcover_path, date = date)

2025-06-19 14:42:42,498 [INFO] Saved FAPAR to data/20230531_00e8dda8/20230531_FAPAR.tif
2025-06-19 14:42:42,707 [INFO] Saved LAI to data/20230531_00e8dda8/20230531_LAI.tif
2025-06-19 14:42:42,896 [INFO] Saved FCOVER to data/20230531_00e8dda8/20230531_FCOVER.tif
2025-06-19 14:42:43,046 [INFO] Saved CCC to data/20230531_00e8dda8/20230531_CCC.tif
2025-06-19 14:42:43,202 [INFO] Saved CWC to data/20230531_00e8dda8/20230531_CWC.tif
2025-06-19 14:42:43,269 [INFO] Saved SCL to data/20230531_00e8dda8/20230531_SCL.tif
2025-06-19 14:42:43,387 [INFO] Saved sunZenithAngles to data/20230531_00e8dda8/20230531_SZA.tif
2025-06-19 14:42:43,435 [INFO] Saved LST to data/20230531_00e8dda8/20230531_LST.tif
2025-06-19 14:42:43,474 [INFO] Saved confidence_in to data/20230531_00e8dda8/20230531_confidence_in.tif
2025-06-19 14:42:43,509 [INFO] Saved viewZenithAngles to data/20230531_00e8dda8/20230531_VZA.tif
2025-06-19 14:42:44,978 [INFO] Saved frac_green to data/20230531_00e8dda8/20230531_F_G.tif
2025-06-19 14:

## 4. Get Meteorological Data
Fetch data from Atmosphere Data Store and Climate Data Store and calculate meteorological featurel

The notebook depends on [Climate Data Store](https://cds.climate.copernicus.eu/) and the [Atmosphere Data Store](https://cds.climate.copernicus.eu/). To access data from these two sources you need an API key as described in the documentation:
* [CDS User Guide](https://cds.climate.copernicus.eu/how-to-api) 

To run the next functions, you need to create a ```.adsapirc``` and a ```.cdsapirc``` file with the API key like this:

``` bash
.adsapirc
url: https://ads.atmosphere.copernicus.eu/api
key: <api_key>
```
and
```` bash
.cdsapirc
url: https://cds.climate.copernicus.eu/api
key: <api_key>
````

In [ ]:
meteo_output_path = get_meteo_data(
    date = str(date),
    bbox = bbox,
    dem_path = dem_s3_path,
    acq_time = sentinel3_acq_time,
    data_dir=dem_s3_path.parent,
    cds_credentials_file=".cdsapirc",
    ads_credentials_file=".adsapirc",
)

2025-06-19 14:42:48,986 [INFO] Processing slope and aspect from DEM.
/usr/lib/python3/dist-packages/osgeo/gdal.py:315: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
2025-06-19 14:42:49,079 [INFO] Process era5 for single date'.


Querying products for extent [55.819327, 7.773194, 53.71218414285715, 10.103551142857127]
..and dates 2023-05-30 00:00:00 to 2023-06-01 00:00:00
Processing ECMWF data for UTC time 2023-05-31 11:00:00
This may take some time...


2025-06-19 14:42:52,310 [INFO] Renaming output files for clarity.
2025-06-19 14:42:52,317 [INFO] Summing DI images.


Saving TA to data/20230531_00e8dda8/meteo_data/20230531T11649_TA.tif
Saving EA to data/20230531_00e8dda8/meteo_data/20230531T11649_EA.tif
Saving WS to data/20230531_00e8dda8/meteo_data/20230531T11649_WS.tif
Saving PA to data/20230531_00e8dda8/meteo_data/20230531T11649_PA.tif
Saving AOT to data/20230531_00e8dda8/meteo_data/20230531T11649_AOT.tif
Saving TCWV to data/20230531_00e8dda8/meteo_data/20230531T11649_TCWV.tif
Saving PAR-DIR to data/20230531_00e8dda8/meteo_data/20230531T11649_PAR-DIR.tif
Saving NIR-DIR to data/20230531_00e8dda8/meteo_data/20230531T11649_NIR-DIR.tif
Saving PAR-DIF to data/20230531_00e8dda8/meteo_data/20230531T11649_PAR-DIF.tif
Saving NIR-DIF to data/20230531_00e8dda8/meteo_data/20230531T11649_NIR-DIF.tif
Saving LW-IN to data/20230531_00e8dda8/meteo_data/20230531T11649_LW-IN.tif
Saving ETr to data/20230531_00e8dda8/meteo_data/20230531_ETR.tif
Saving SW-IN-DD to data/20230531_00e8dda8/meteo_data/20230531_SW-IN-DD.tif


2025-06-19 14:42:52,459 [INFO] Final output saved to: data/20230531_00e8dda8/meteo_data/20230531T11649_S_dn.tif


## 5. Resample Meteorological to same resolution as Sentinel2

In [ ]:
lai_path = Path(tif_path)/f"{str(date).replace('-', '')}_LAI.tif"
resample_to_s2(meteo_output_path, meteo_output_path, lai_path, product_list=["EA.tif", "p.tif", "u.tif", "S_dn_24.tif", "S_dn.tif", "T_A1.tif"])

2025-06-19 14:42:56,847 [INFO] Starting resample_s2 processing...
2025-06-19 14:42:56,907 [INFO] Using Sentinel-2 extent: (485380.0, 6063050.0, 506620.0, 6074720.0)
2025-06-19 14:42:56,909 [INFO] Processing 20230531T11649_S_dn.tif -> data/20230531_00e8dda8/meteo_data/20230531T11649_S_dn.tif
2025-06-19 14:42:57,069 [INFO] Processing 20230531T11649_EA.tif -> data/20230531_00e8dda8/meteo_data/20230531T11649_EA.tif
2025-06-19 14:42:57,177 [INFO] Processing 20230531T11649_p.tif -> data/20230531_00e8dda8/meteo_data/20230531T11649_p.tif
2025-06-19 14:42:57,301 [INFO] Processing 20230531T11649_T_A1.tif -> data/20230531_00e8dda8/meteo_data/20230531T11649_T_A1.tif
2025-06-19 14:42:57,454 [INFO] Processing 20230531T11649_u.tif -> data/20230531_00e8dda8/meteo_data/20230531T11649_u.tif
2025-06-19 14:42:57,612 [INFO] Processing 20230531_S_dn_24.tif -> data/20230531_00e8dda8/meteo_data/20230531_S_dn_24.tif
2025-06-19 14:42:57,766 [INFO] Processing completed successfully.
2025-06-19 14:42:57,768 [INFO